1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


**hit_rate@k** - был ли хотя бы один релевантный товар среди топ k рекомендованных. Как правило недостаточно информативна, т.к. принимает только значения 1/0.

**precision@k** - доля релевантных товаров, среди топ k рекомендованных.

На мой взгляд, метрика hit_rate@k может быть актуальна, если мы рассматриваем аналогичные товары, среди которых наш клиент купит лишь 1, например при покупке квартиры или поиске подходящей вакансии.

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Логарифм используется как функция дисконтирования: ранжирование позиций в начале списка отличаются гораздо сильнее, чем позиции в его конце. 
Для этой цели так же можно использовать степенные функции.

Логарифм так же используется в logloss, TF-IDF.

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

Можно так же использовать F-score, для балансирования показателей recall и precision, корреляцию Спирмана.

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [1]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

In [2]:
bought = [1, 3, 5, 7, 9, 11]  
recommended_8 = [2, 5, 7, 4, 11, 9, 8, 10] 
k = 8

In [3]:
TP = [5, 7, 9, 11]
FP = [2, 4, 8, 10]
FN = [1, 3]

In [4]:
def precision_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [5]:
precision_man = len(TP)/(len(TP) + len(FP))
precision = precision_at_k(recommended_8, bought, k)

print(f'pr@8: {precision_man} / {precision}')

pr@8: 0.5 / 0.5


In [6]:
def recall_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [7]:
recall_man = len(TP)/(len(TP) + len(FN))
recall = recall_at_k(recommended_8, bought, k)

print(f'rec@8: {recall_man} / {recall}')

rec@8: 0.6666666666666666 / 0.6666666666666666


In [8]:
def ap_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 

        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)

            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [9]:
average_precision_man = 1/k*(0+0+2/3+2/4+3/5+4/6)
average_precision = ap_k(recommended_8, bought, k)

print(f'AP@8: {average_precision_man} / {average_precision}')

AP@8: 0.30416666666666664 / 0.30416666666666664


In [10]:
def dcg_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if flags[0] == True:
        sum_ = 1
    else:
        sum_ = 0
        
    for i in range(2, k+1): 
        if flags[i-1] == True:
            log_2 = 1 / np.log2(i)
            sum_ += log_2
            
    return sum_ / k
    
    
def ndcg_at_k(recommended_list, bought_list, k):
    
    sum_ = 1
    
    for i in range(2, k+1):
        log_2 = 1 / np.log2(i)
        sum_ += log_2
        
    ideal_dcg = sum_ / k
    
    dcg = dcg_at_k(recommended_list, bought_list, k)
    
    return dcg / ideal_dcg

In [11]:
dcg_man = 1/8 * (1/np.log2(2) + 1/np.log2(3) + 1/np.log2(5) + 1/np.log2(6))
ideal_dcg = 1/8 * (1/1 + 1/np.log2(2) + 1/np.log2(3) + 1/np.log2(4) + 1/np.log2(5) + 1/np.log2(6) + 1/np.log2(7) + 1/np.log2(8))
ndcg_man = dcg_man / ideal_dcg

ndcg = ndcg_at_k(recommended_8, bought, k)

print(f'NDCG@8: {ndcg_man} / {ndcg}')

NDCG@8: 0.5279127445637272 / 0.5279127445637272


In [12]:
def reciprocal_rank(recommended_list, bought_list):
    ranks=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [13]:
mrr_man = (1/2+1/3+1/5+1/6)/8
mrr = reciprocal_rank(recommended_8, bought)

print(f'MRR@8: {mrr_man} / {mrr}')

MRR@8: 0.15 / 0.15
